### Import Libraries

In [14]:
import pandas as pd 
import numpy as np
import enum
import io
import json
import os
import re
import sys
import json
import xgboost as xgb
import lightgbm as lgb
import nltk
import mlflow
import seaborn as sns
import matplotlib.pyplot as plt
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.layers import Dense, Conv2D, Embedding, Dropout, Conv1D, GlobalMaxPooling1D, LSTM, Bidirectional
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.preprocessing.sequence import pad_sequences
# from keras.preprocessing.text import Tokenizer
# from keras.models import Sequential, load_model
# from keras import layers, Input, Model
from tqdm import tqdm
tqdm.pandas()
from googletrans import Translator
from sklearn import preprocessing
from sklearn.utils import resample, shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import *
from imblearn.pipeline import Pipeline as imbPipeline
from imblearn.ensemble import BalancedRandomForestClassifier
from classes.Accuracy import Accuracy
from classes.CustomTokenizer import *
from classes.VectEnum import *

C:\ProgramData\Anaconda3\envs\env-fsadosky\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# # Model constants.
# max_features = 20000
# embedding_dim = 128
# sequence_length = 500
# vocab_size = 30982
# max_length = 5890#len(tfidf_vect.get_feature_names())
# 
# # define baseline model
# def baseline_model():
#     # create model
#     model = Sequential()
#     model.add(Embedding(vocab_size, 128, trainable = False))
#     model.add(Bidirectional(LSTM(128)))
# #   model.add(LSTM(128))
#     model.add(Dense(32, activation = "relu"))
#     model.add(Dropout(0.5))
#     model.add(Dense(352, activation = "softmax"))
#     model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])    
#     return model

In [16]:
# #--------------------------------------------------#
# ### TO USE STACKING OR VOTING UNCOMMENT ALL THIS ###
# #--------------------------------------------------#
# # define the base models
# level0 = list()	
# level0.append(('knc', KNeighborsClassifier()))
# level0.append(('lsvc', LinearSVC()))
# level0.append(('svm', SVC(C=10, gamma=0.1)))	
# # define meta learner model
# level1 = LogisticRegression()
# # define the stacking ensemble
# classifier = StackingClassifier(estimators=level0, final_estimator=level1, verbose=2)
# #classifier = VotingClassifier(estimators=level0, verbose=2)

In [17]:
# #--------------------------------------------------#
# ###                  PARAMETERS                  ###
# #--------------------------------------------------#

APPLY_RESAMPLE = False
VECTORIZER_TYPE = VectEnum.TfidfVectorizer
RANDOM_STATE = 42
TEST_SIZE = 0.2
MIN_DF = 1            
TOKENIZER_TYPE = None #custom_preprocess | nltk.word_tokenize
SHUFFLE = True
SAMPLING_STRATEGY = 'not majority'
N_JOBS = -1
K_NEIGHBORS = 3
SCORING = 'balanced_accuracy'
REFIT = 'balanced_accuracy'
CV = 5

In [18]:
# #--------------------------------------------------#
# ###                  CLASSIFIERS                 ###
# #--------------------------------------------------#

classifier_list = []
classifier_list.append(xgb.XGBClassifier(n_jobs=-1, verbose=2)) 
classifier_list.append(SVC(verbose=2)) 
classifier_list.append(LinearSVC(verbose=2, random_state=42)) 
classifier_list.append(RandomForestClassifier(n_jobs=-1, verbose=2)) 
classifier_list.append(BalancedRandomForestClassifier(n_jobs=-1, verbose=2)) 
classifier_list.append(KNeighborsClassifier(n_jobs=-1)) 
classifier_list.append(lgb.LGBMClassifier()) 
classifier_list.append(DecisionTreeClassifier()) 
classifier_list.append(LogisticRegression(class_weight='balanced', n_jobs=-1))
classifier_list.append(BalancedRandomForestClassifier())

oversample = SMOTE(random_state=RANDOM_STATE, n_jobs=N_JOBS, sampling_strategy=SAMPLING_STRATEGY, k_neighbors=K_NEIGHBORS)         

### Helper Code

In [19]:
def add_mock_data(df, category, multiplier=10):    
    mask = (df['Intencion_cat_label'] == category)
    row = df[mask]
    return df.append([row]*multiplier,ignore_index=True)

def apply_resample(df, column, num_cases=5, multiplier=10):
    print('-----------------')
    print('Before Resample: ')
    print('-----------------')
    print('Train Shape: ' + str(df.shape))
    
    #add another row to minority class with few values
    grouped = df.groupby('Intencion_cat_label').count().sort_values(by='Intencion_cat_label', ascending=True)
    poor_cases = grouped[grouped[column] < num_cases]
    print('Poor cases: {}'.format(len(poor_cases)))
    print(poor_cases)
    for index in poor_cases.index.unique():
        df = add_mock_data(df, index, multiplier)

    print('----------------')
    print('After Resample: ')
    print('----------------')
    print('Train Shape: ' + str(df.shape))
    return df

def __Print(text):
    print('')
    print('--------------------------------------------------------')
    print('-- {} --'.format(text))
    print('--------------------------------------------------------')        


def GetVectorizer():        
    if VECTORIZER_TYPE.name == 'TfidfVectorizer':
        return TfidfVectorizer(min_df=MIN_DF, tokenizer=TOKENIZER_TYPE)
    elif VECTORIZER_TYPE.name == 'CountVectorizer':
        return CountVectorizer(min_df=MIN_DF, tokenizer=TOKENIZER_TYPE)            

def GetPipeline():
    vect = GetVectorizer()
    imbpipe = imbPipeline(steps=[('vect', vect), ('resample', oversample), ('clf', classifier)], verbose=2)
    pipeline = Pipeline(steps=[('vect', vect), ('clf', classifier)], verbose=2)
    return imbpipe if (APPLY_RESAMPLE == True) else pipeline

def Summary(model, model_name, X_train, X_test, y_train, y_test):        
    __Print('Summary')
    training_score = model.score(X_train , y_train)
    test_score = model.score(X_test  , y_test )
    mlflow.log_metric('training_score', training_score)
    mlflow.log_metric('test_score', test_score)
    print("Training set score for " + model_name + " %f" % training_score)
    print("Testing  set score for " + model_name + " %f" % test_score)
    print('--------------------------------------------------------')        

def SaveBestParamsToDisk(model_name, model_best_params):                
    print("Best Params for " + model_name + ": " + str(model_best_params))
    filename = ''.join(['model_best_params/', model_name, '_best_params.json'])
    __Print('Saving Best Parameters for {} on {}'.format(model_name, filename))
    with open(filename, 'w') as outfile:
        json.dump(model_best_params, outfile)        

def GetModelParams(model_name):
    filename = 'model_params/{}_params.json'.format(model_name)
    fileexists = os.path.isfile(filename)
    if fileexists == False:
        with io.open(filename, 'w') as json_file:
            json_file.write(json.dumps({}))
    with open(filename) as json_file:
        params_grid = json.load(json_file)
    return params_grid    

def GenerateTrainedModel(classifier, X_train, X_test, y_train, y_test):
    model_name = classifier.__class__.__name__
    params_grid = GetModelParams(model_name)                                    
    print('Preprocessing data...')        
    pipeline = GetPipeline()         
    gridsearch = GridSearchCV(pipeline, params_grid, cv=CV, n_jobs=N_JOBS, verbose=2, scoring=SCORING, refit=REFIT)
    print('Training Model... {}'.format(model_name))
    gridsearch.fit(X_train, y_train)                
    optimized_model = gridsearch.best_estimator_
    print('Finished Training Model.')        
    model_best_params = gridsearch.best_params_                
    SaveBestParamsToDisk(model_name, model_best_params)
    Summary(optimized_model, model_name, X_train, X_test, y_train, y_test)        
    return optimized_model, model_best_params, X_train, X_test, y_train, y_test

def TrainModel(X, y, classifier):    
    pipeline = GetPipeline()
    print('Training Model With all train dataset...')
    pipeline.fit(X, y)
    print('Finished Training Model With all train dataset.')
    __Print('Summary')
    training_score = pipeline.score(X , y)
    print("Training score for all data: %f" % training_score)
    return pipeline

### Get Info from CSV

In [20]:
df_train = shuffle(pd.read_csv('data/train_preprocessed.csv', sep='|'))
df_test = shuffle(pd.read_csv('data/test_santander.csv', usecols=['id','Pregunta']))

In [21]:
print(df_train['Intencion_cat_label'].value_counts())
# add one more sample because I have one case with just one sample and stratify need at least 2 samples
df_train = apply_resample(df_train, 'Pregunta', 2, 1)

132    600
293    567
1      493
147    488
223    477
      ... 
11       3
15       3
13       3
24       2
104      1
Name: Intencion_cat_label, Length: 352, dtype: int64
-----------------
Before Resample: 
-----------------
Train Shape: (20104, 7)
Poor cases: 1
                     Pregunta  Intencion  \
Intencion_cat_label                        
104                         1          1   

                     Preguntas_custom_preprocess_no_stopwords  \
Intencion_cat_label                                             
104                                                         1   

                     Preguntas_custom_preprocess  \
Intencion_cat_label                                
104                                            1   

                     Preguntas_custom_preprocess_w_verbs  \
Intencion_cat_label                                        
104                                                    1   

                     Preguntas_word_tokenize  
Intencion_cat_label 

### train_test_split

In [22]:
X = df_train['Preguntas_word_tokenize'].values
y = df_train['Intencion_cat_label'].values

X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size = TEST_SIZE, stratify=y, random_state=RANDOM_STATE, shuffle=SHUFFLE)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

C:\ProgramData\Anaconda3\envs\env-fsadosky\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
if APPLY_RESAMPLE == True:
    #Join X_train and y_train to add more classes to train

    df_Xtrain = pd.DataFrame(X_train, columns=['Pregunta'])
    df_Xtrain.set_index('Pregunta')
    df_Xtrain['Intencion_cat_label'] = y_train
    cnt_pro = df_Xtrain['Intencion_cat_label'].value_counts()

    plt.figure(figsize=(35,4))
    sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
    plt.ylabel('Number of Occurrences', fontsize=12)
    plt.xlabel('Intencion', fontsize=12)
    plt.xticks(rotation=90)
    plt.show()

    #Apply resample to train dataset
    df_Xtrain = apply_resample(df_Xtrain, 'Pregunta', 50, 10)

    X_train = df_Xtrain['Pregunta'].values
    y_train = df_Xtrain['Intencion_cat_label'].values

C:\ProgramData\Anaconda3\envs\env-fsadosky\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
for classifier in classifier_list:
    #try:
        with mlflow.start_run():
            mlflow.log_param('Classifier', classifier.__class__.__name__)    
            mlflow.log_param('resampling', APPLY_RESAMPLE)         
            mlflow.log_param('vect', VECTORIZER_TYPE.name)
            mlflow.log_param('tokenizer', ('None' if (TOKENIZER_TYPE == None) else TOKENIZER_TYPE.name))    
            mlflow.log_param('resampling_class', oversample.__class__.__name__) 
            mlflow.log_param('sampling_strategy', SAMPLING_STRATEGY)   

            optimized_model, model_best_params, X_train, X_test, y_train, y_test = GenerateTrainedModel(classifier, X_train, X_test, y_train, y_test)
            pred = optimized_model.predict(X_test)
            #Compute the balanced accuracy
            #The balanced accuracy in binary and multiclass classification problems to deal with imbalanced datasets. It is defined as the average of recall obtained on each class.
            #The best value is 1 and the worst value is 0 when adjusted=False.
            print('Calculating Scores...')
            balanced_accuracy_score = Accuracy.get_balanced_accuracy_score(y_test, pred)
            accuracy_score = Accuracy.get_accuracy_score(y_test, pred)
            
            print('Train best model with all train data')
            optimized_model = TrainModel(X, y, classifier)
            
            print('Processing Classification Report...')
            report = Accuracy.get_classification_report(y_test, pred)
            df_report = pd.DataFrame()
            lines = report.split('\n')
            for line in lines[2:-5]:    
                row_data = line.split('      ')             
                row = pd.Series(data={'class': row_data[1].strip(), 
                                        'precision': row_data[2].strip(), 
                                        'recall': row_data[3].strip(), 
                                        'f1-score': row_data[4].strip(), 
                                        'support': row_data[5].strip()
                                        }
                                        )
                df_report = df_report.append(row, ignore_index=True)
            df_report['f1-score'] = df_report['f1-score'].astype(float)
            df_report['precision'] = df_report['precision'].astype(float)
            df_report['recall'] = df_report['recall'].astype(float)
            df_report['class'] = df_report['class'].astype(int)
            
            print('Save predictions of test to CSV...')
            pred_det = optimized_model.predict(df_test['Pregunta'].values)
            df_test['Intencion'] = pred_det
            SUBMIT_FILE = 'data/submit_{}.csv'.format(classifier.__class__.__name__)
            df_test.to_csv(SUBMIT_FILE, mode='w', header=False, columns=['id','Intencion'], index=False, sep=',')

            print('Logging metrics...')
            #Save metrics
            mlflow.log_metric("balanced_accuracy_score", balanced_accuracy_score)
            mlflow.log_metric("accuracy_score", accuracy_score)                
            for key, value in model_best_params.items():        
                mlflow.log_param(key, value)
            mlflow.sklearn.log_model(optimized_model, "model")
            mlflow.log_metric("classification_report", df_report)                
    #except:
    #    print('There was an error!')
    #    print(sys.exc_info())
    #    mlflow.end_run()
    #    continue

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Preprocessing data...
Training Model...
Fitting 5 folds for each of 1 candidates, totalling 5 fits


### Classification Report

In [12]:
cnt_pro = df_report[df_report['recall'] < 0.50]

fig, ax = plt.subplots(nrows=2, sharex=True)
fig.set_figheight(5)
fig.set_figwidth(15)
sns.barplot(cnt_pro.index, cnt_pro.recall, alpha=0.8, ax=ax[0])
plt.ylabel('Recall', fontsize=12)
plt.xlabel('Intencion', fontsize=12)

items = df_train[df_train['Intencion_cat_label'].isin(cnt_pro.index)]['Intencion_cat_label'].value_counts()

sns.barplot(items.index, items.values, alpha=0.8, ax=ax[1])
plt.ylabel('Number of Occurrences', fontsize=12)

plt.xticks(rotation=90)

plt.show()

C:\ProgramData\Anaconda3\envs\env-fsadosky\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df_report' is not defined

In [13]:
poor_values = df_train[df_train.Intencion_cat_label == 337]
poor_values

,Pregunta,Intencion,Intencion_cat_label,Preguntas_custom_preprocess_no_stopwords,Preguntas_custom_preprocess,Preguntas_custom_preprocess_w_verbs,Preguntas_word_tokenize
29,solicitud tarjeta credito visa,Cat_337,337,"['solicitud', 'tarjet', 'credit', 'vis']","['solicitud', 'tarjet', 'credit', 'vis']","['solicitud', 'tarjet', 'credit', 'vis']","['solicitud', 'tarjeta', 'credito', 'visa']"
39,quiera solicitar la tarjeta santander rio,Cat_337,337,"['quier', 'solicit', 'la', 'tarjet', 'santand'...","['quier', 'solicit', 'tarjet', 'santand', 'rio']","['quier', 'solicit', 'tarjet', 'santand', 'rio']","['quiera', 'solicitar', 'la', 'tarjeta', 'sant..."
62,como obtengo mi primera tarjeta credito,Cat_337,337,"['com', 'obteng', 'mi', 'primer', 'tarjet', 'c...","['obteng', 'primer', 'tarjet', 'credit']","['obteng', 'primer', 'tarjet', 'credit']","['como', 'obtengo', 'mi', 'primera', 'tarjeta'..."
151,solicitar tarjeta de credito,Cat_337,337,"['solicit', 'tarjet', 'de', 'credit']","['solicit', 'tarjet', 'credit']","['solicit', 'tarjet', 'credit']","['solicitar', 'tarjeta', 'de', 'credito']"
196,cobro jubilacion tarjeta,Cat_337,337,"['cobr', 'jubilacion', 'tarjet']","['cobr', 'jubilacion', 'tarjet']","['cobr', 'jubilacion', 'tarjet']","['cobro', 'jubilacion', 'tarjeta']"
...,...,...,...,...,...,...,...
19666,quiero tarjeta mastercard santander,Cat_337,337,"['quier', 'tarjet', 'mastercard', 'santand']","['quier', 'tarjet', 'mastercard', 'santand']","['quer', 'tarjet', 'mastercard', 'santand']","['quiero', 'tarjeta', 'mastercard', 'santander']"
19763,necesito adquirir tarjeta visa,Cat_337,337,"['necesit', 'adquir', 'tarjet', 'vis']","['necesit', 'adquir', 'tarjet', 'vis']","['necesit', 'adquir', 'tarjet', 'vis']","['necesito', 'adquirir', 'tarjeta', 'visa']"
19820,quiero tarjeta santander,Cat_337,337,"['quier', 'tarjet', 'santand']","['quier', 'tarjet', 'santand']","['quer', 'tarjet', 'santand']","['quiero', 'tarjeta', 'santander']"
20041,quisiera saber si puedo tener mi tarjeta de cr...,Cat_337,337,"['quis', 'sab', 'si', 'pued', 'ten', 'mi', 'ta...","['quis', 'sab', 'pued', 'ten', 'tarjet', 'cred...","['quis', 'sab', 'pued', 'ten', 'tarjet', 'cred...","['quisiera', 'saber', 'si', 'puedo', 'tener', ..."
